## 1.链表环问题
给定两个可能有环也可能无环的单链表，头节点head1、head2。实现函数返回相交的第一个节点或不相交返回None
* 要求时间O(N)，空间O(1)
* 情况1：链表1无环、 链表2无环。见1.2
* 情况2：链表1有环、 链表2无环。一定不相交
* 情况3：链表1无环、 链表2有环。一定不相交
* 情况4：链表1有环、 链表2有环。见1.3

### 1.1 单链表返回环的第一个节点或无环返回None
* 快走2，慢走1；快前面空了，无环
* 有环，快慢一定相遇，但相遇处不一定为入环节点；
* 相遇后，快节点回到head，快慢此后同时走一步，最终会在环交点相遇

In [1]:
#找到入环节点
def getLoopNode(head):
    if not head or not head.next or not head.next.next:
        return None
    n1 = head.next
    n2 = head.next.next
    while n1 != n2:
        if not n2.next or not n2.next.next:
            return None
        n2 = n2.next.next
        n1 = n1.next
    n2 = head
    while n1 != n2:
        n1 = n1.next
        n2 = n2.next
    return n1

### 1.2 两个无环单链表，判断是否相交
* 分别找到两个单链表的最后一个节点，如果这两个节点不是一个节点（地址不同），一定不相交
* 长链表长度a，短链表长度b。让短链表先走a-b，然后长短一起走，第一个相同的节点相遇

In [3]:
def noLoop(head1, head2):
    if not head1 or not head2:
        return None
    cur1 = head1
    cur2 = head2
    n = 0
    while cur1.next:
        n += 1
        cur1 = cur1.next
    #此时cur1走到最后节点
    while cur2.next:
        n -= 1
        cur2 = cur2.next
    if cur1 != cur2:
        return None
    #一定存在交点
    if n > 0: #谁长cur1
        cur1 = head1 
        cur2 = head2
    else:
        cur1 = head2
        cur2 = head1
    n = abs(n)
    while n != 0:
        n -= 1
        cur1 = cur1.next
    while cur1 != cur2:
        cur1 = cur1.next
        cur2 = cur2.next
    return cur1

### 1.3 两个都有环的单链表，判断是否相交
* 情况4.1： 不相交，各自成环。
* 情况4.2： 相交，相交后成环（入环节点同一个）。 == 两个链表的getLoopNode的两个入环点相同。 == 两个无环链表(入环节点为截止)相交问题
* 情况4.3： 相交，先成环，后相交（入环节点非同一个节点）（脑袋上俩犄角）
* 如果两个入环点不相同！命中情况1、情况3：从一个入环节点开始走，如果遇到自己前没遇到另一个入环节点，命中情况4.1；反之情况4.3

In [2]:
def bothLoop(head1, loop1, head2, loop2):
    cur1 = None
    cur2 = None
    #情况2
    if loop1 == loop2:
        cur1 = head1
        cur2 = head2
        n = 0
        while cur1 != loop1:
            n += 1
            cur1 = cur1.next
        while cur2 != loop2:
            n -= 1
            cur2 = cur2.next
        if n > 0:
            cur1 = head1
            cur2 = head2
        else:
            cur1 = head2
            cur2 = head1
        n = abs(n)
        while n!=0:
            n -= 1
            cur1 = cur1.next
        while cur1 != cur2:
            cur1 = cur1.next
            cur2 = cur2.next
        return cur1
    else:
        cur1 = loop1.next
        while cur1 != loop1:
            #情况3
            if cur1 == loop2:
                return loop1
            cur1 = cur1.next
        #情况1 cur1 == loop1
        return None

In [4]:
# 主函数
def getIntersectNode(head1, head2):
    if not head1 or not head2:
        return None
    loop1 = getLoopNode(head1)
    loop2 = getLoopNode(head2)
    if not loop1 and not loop2:
        return noLoop(head1, head2)
    if loop1 and loop2:
        return bothLoop(head1, loop1, head2, loop2)
    return None

## 2.二叉树
* 先序：中左右
* 中序：左中右
* 后序：左右中

### 2.1 求x的所有祖先节点
已知先序、后序数组。x在先序的所有左节点 ∩ x在后序的所有右节点 = x的所有祖先节点
* 证明：
    1. 先序是头左右，那么x的祖先一定在左； 后序是左右头，x的祖先一定在x的右侧。所以左+右一定包含所有祖先
    2. 为什么交集一定是祖先节点且不包括其他节点：(3~5)
    3. x的所有孩子只会出现在先序的右侧，所以不包含所有子节点
    4. x挂在祖先的左子树上，祖先下面的右子树所有节点，只会出现在先序x的右侧，不会出现在先序x的左侧
    5. x挂在祖先的右子树上，祖先下面的左子树所有节点，虽然会出现在先序x的左侧，但不会出现在后序x的右侧

### 2.2非递归方式实现二叉树的先序、中序、后序遍历

#### 2.2.1 先序
* 栈顶弹出cur
* 有右压右，有左压左

In [5]:
def preOrder(head):
    print("pre-order: ")
    if head:
        stack = list()
        stack.append(head)
        while stack:
            cur = stack.pop()
            print(cur.val, end=" ")
            if cur.right:
                stack.append(cur.right)
            if cur.left:
                stack.append(cur.left)
    print("done.")

#### 2.2.2 后序
* 栈顶弹出cur
* 有左压左，有右压右
* 相对先序，得到中、右、左。弹出时不打印，压入另一个打印栈，从打印栈依次弹出时打印

In [6]:
def postOrder(head):
    print("post-order: ")
    if head:
        stack = list()
        stack.append(head)
        printStack = list()
        while stack:
            cur = stack.pop()
            printStack.append(cur)#压入另一个打印栈
            if cur.left:
                stack.append(cur.left)
            if cur.right:
                stack.append(cur.right)
    #从打印栈依次弹出时打印
    while printStack:
        print(printStack.pop().val, end=" ")
    print("done.")

In [1]:
#只用一个栈实现
def postOrder2(head):
    print("post-order: ")
    if not head:
        stack = list()
        stack.append(head)
        cur = None
        while stack:
            cur = stack[-1]
            if cur.left and head != cur.left and head != cur.right:
                stack.append(cur.left)
            elif cur.right and head != cur.right:
                stack.append(cur.right)
            else:
                print(stack.pop().val, end=" ")
                head = cur
    print("done.")

#### 2.2.3 中序
* 当前节点cur，cur为头的数，整条左边界进栈（cur压入，cur=cur.left，till cur==None)
* 从栈中弹出节点打印，打印后将弹出的节点的右孩子设为cur。重复上面过程

In [7]:
def inOrder(head):
    print("in-order: ")
    if not head:
        stack = list()
        cur = head
        while stack or cur:
            if cur:
                stack.append(cur)
                cur = cur.left
            else:
                cur = stack.pop()
                print(cur.val, end=" ")
                cur = cur.right
                
    print("done.")